In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset

In [2]:
import matplotlib.pyplot as plt

In [3]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"
out_folder = "/home/mlopez/EXEC/TimeMeansModels/"

# Select variable

In [4]:
#Variable 
var="tg_mean"
#var ="tn_mean"
#var ="tx_mean"
#var ="solidprcptot"
#var ="prcptot"
#var ="dlyfrzthw"
#var ="growing_season_length"
#var ="DJC"


files85 = glob.glob(folder+"*"+"rcp85_"+var+"_annual.nc")
files45 = glob.glob(folder+"*"+"rcp45_"+var+"_annual.nc")


filesSeas45 =  glob.glob(folder+"*rcp45_"+var+"_seasonal.nc")
filesSeas85 =  glob.glob(folder+"*rcp85_"+var+"_seasonal.nc")



# FOR RCP 45

In [5]:
dsEns45= ens.create_ensemble(files45)
#dsEns45

## Getting time windows and percentiles

In [6]:
# Table 
# Mapping data - 30 year means
tmp1 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
window = 30
time1 = tmp1.time[0::window]
# 30 y means
ds30yavgTable = tmp1.coarsen(time=window).mean()
ds30yavgTable['time'] = time1
# spatial mean
perc30yavgTable = ens.ensemble_percentiles(ds30yavgTable)
#perc30yavgTable 

In [7]:
%time dfperT45 = perc30yavgTable.drop('realization').to_dataframe().dropna()

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



CPU times: user 52.2 s, sys: 3min 31s, total: 4min 23s
Wall time: 5.16 s


In [8]:
dfperT45['season'] = "annual"

In [9]:
dfperT45

tg_mean_p10  tg_mean_p50  tg_mean_p90  season
lat       lon        time                                                     
66.623306 -89.045212 1981-01-01   259.535767   259.757019   259.995667  annual
                     2011-01-01   260.684479   261.159607   261.800842  annual
                     2041-01-01   261.750183   262.530670   263.948517  annual
                     2071-01-01   262.040283   263.137421   264.745941  annual
          -88.961884 1981-01-01   259.636353   259.858429   260.096039  annual
...                                      ...          ...          ...     ...
40.041039 -74.129143 2071-01-01   286.643341   287.808899   288.279480  annual
          -74.045807 1981-01-01   285.045471   285.300079   285.372498  annual
                     2011-01-01   285.980835   286.404816   286.786285  annual
                     2041-01-01   286.491486   287.340820   287.734955  annual
                     2071-01-01   286.677124   287.829163   288.284882  annual

[304824 rows x 4 columns]

# FOR RCP85

In [10]:
dsEns85= ens.create_ensemble(files85)
#dsEns85

## Getting time windows and percentiles

In [11]:
#for (file, percentile, season) in ((files45, 45, "annual"), (files85, 85, "annual"),
#                                  (filesSeas45, 45, )):
# Table 
# Mapping data - 30 year means
tmp1 = dsEns85.sel(time=(dsEns85.time.dt.year>=1981))
window = 30
time1 = tmp1.time[0::window]
# 30 y means
ds30yavgTable = tmp1.coarsen(time=window).mean()
ds30yavgTable['time'] = time1
# spatial mean
perc30yavgTable85 = ens.ensemble_percentiles(ds30yavgTable)
#perc30yavgTable 

%time dfperT85 = perc30yavgTable85.drop('realization').to_dataframe().dropna()
dfperT85['season'] = "annual"

dfperT85

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



CPU times: user 44.9 s, sys: 3min 57s, total: 4min 42s
Wall time: 5.65 s


tg_mean_p10  tg_mean_p50  tg_mean_p90  season
lat       lon        time                                                     
66.623306 -89.045212 1981-01-01   259.534363   259.676971   259.979248  annual
                     2011-01-01   260.907867   261.137909   262.152130  annual
                     2041-01-01   262.849518   263.773621   265.617798  annual
                     2071-01-01   264.740723   267.102417   269.068146  annual
          -88.961884 1981-01-01   259.635101   259.778412   260.079498  annual
...                                      ...          ...          ...     ...
40.041039 -74.129143 2071-01-01   288.501953   290.123566   290.468079  annual
          -74.045807 1981-01-01   285.091705   285.309387   285.382385  annual
                     2011-01-01   286.110199   286.495880   286.829041  annual
                     2041-01-01   287.208649   288.221466   288.346680  annual
                     2071-01-01   288.534546   290.137482   290.489441  annual

[304824 rows x 4 columns]

# FOR Seasonal RCP45

## Loop to create all 4 dataframes

In [124]:
def load_seasonal_data(files, scenario):
    dsEns= ens.create_ensemble(files)
    dsEns.time.dt.season[0:4]
    # Seasonal example 30 y means
    listds = []
    i=0
    for s in dsEns.time.dt.season[0:4]:
        print (i)
        tmp1 = dsEns.sel(time=(dsEns.time.dt.year>=1981))
        tmp1 = tmp1.sel(time=(tmp1.time.dt.season==s))
        window = 30
        time1 = tmp1.time[0::window]
        # 30 y means
        ds30yavgTable = tmp1.coarsen(time=window).mean()
        ds30yavgTable['time'] = time1
        # percentiles
        perc30yavgTable = ens.ensemble_percentiles(ds30yavgTable)
        #perc30yavgTable 
        dfSeas = perc30yavgTable.drop('realization').to_dataframe().dropna()
        if i== 0: season = "winter" 
        elif i == 1: season = "spring"
        elif i == 2: season = "summer"
        elif i == 3: season = "fall"
        print (season)
        dfSeas["season"] = season
        print(dfSeas)
        dfSeas["scenario"] = scenario        
        i = i+1
        listds.append(dfSeas)
    return pd.concat(listds)

## To split by region FOR RCP45 AND RCP85, Annual and seasonal

In [18]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [19]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [20]:
 def strings_to_nan(col):
    return col.map(lambda x: np.nan if isinstance(x, str) else x)

In [21]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

In [125]:
dfperT45["scenario"] = 45
dfperT85["scenario"] = 85

In [160]:
dfbig = pd.concat((load_seasonal_data(filesSeas45, 45), load_seasonal_data(filesSeas85,  85),
                   dfperT45, dfperT85))
dfbig = dfbig.reset_index()
dfbig["time"] = dfbig.time.dt.year

0


  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



winter
                                 tg_mean_p10  tg_mean_p50  tg_mean_p90  season
lat       lon        time                                                     
66.623306 -89.045212 1981-12-01   242.248459   242.892715   243.215439  winter
                     2011-12-01   243.738480   244.853302   245.544235  winter
                     2041-12-01   245.134262   247.326920   248.365189  winter
                     2071-12-01   245.974152   247.535126   250.209167  winter
          -88.961884 1981-12-01   242.332092   242.979950   243.297638  winter
...                                      ...          ...          ...     ...
40.041039 -74.129143 2071-12-01   275.978424   276.909088   277.717651  winter
          -74.045807 1981-12-01   274.389282   274.760345   275.065002  winter
                     2011-12-01   274.996582   275.642120   276.623352  winter
                     2041-12-01   275.384796   276.625397   277.195892  winter
                     2071-12-01   276.045074 

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



spring
                                 tg_mean_p10  tg_mean_p50  tg_mean_p90  season
lat       lon        time                                                     
66.623306 -89.045212 1981-03-01   254.374466   254.748276   254.991730  spring
                     2011-03-01   255.154022   255.703262   256.806183  spring
                     2041-03-01   256.143494   256.853729   257.824738  spring
                     2071-03-01   256.383209   258.134460   258.608734  spring
          -88.961884 1981-03-01   254.491623   254.866119   255.108704  spring
...                                      ...          ...          ...     ...
40.041039 -74.129143 2071-03-01   285.295227   285.980774   286.553284  spring
          -74.045807 1981-03-01   283.247925   283.736969   283.930511  spring
                     2011-03-01   284.211090   284.595367   284.985931  spring
                     2041-03-01   284.660309   285.588745   285.786407  spring
                     2071-03-01   285.256195 

...

spring
                                 tg_mean_p10  tg_mean_p50  tg_mean_p90  season
lat       lon        time                                                     
66.623306 -89.045212 1981-03-01   254.457214   254.777252   255.002594  spring
                     2011-03-01   254.998367   256.012848   257.180634  spring
                     2041-03-01   256.893005   258.035553   258.763763  spring
                     2071-03-01   259.115143   260.579132   262.208954  spring
          -88.961884 1981-03-01   254.573090   254.899719   255.118149  spring
...                                      ...          ...          ...     ...
40.041039 -74.129143 2071-03-01   286.450623   287.730621   288.091980  spring
          -74.045807 1981-03-01   283.467957   283.717102   283.952667  spring
                     2011-03-01   284.383759   284.824677   285.028595  spring
                     2041-03-01   285.397400   286.214722   286.484375  spring
                     2071-03-01   286.427155 

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



summer
                                 tg_mean_p10  tg_mean_p50  tg_mean_p90  season
lat       lon        time                                                     
66.623306 -89.045212 1981-06-01   278.095978   278.492523   278.602356  summer
                     2011-06-01   278.526031   279.476196   280.333862  summer
                     2041-06-01   279.445679   280.894348   282.732544  summer
                     2071-06-01   280.813019   284.002625   286.054413  summer
          -88.961884 1981-06-01   278.180664   278.577942   278.687103  summer
...                                      ...          ...          ...     ...
40.041039 -74.129143 2071-06-01   299.188263   301.165375   301.748016  summer
          -74.045807 1981-06-01   295.787292   295.834747   295.985748  summer
                     2011-06-01   296.817261   297.001343   297.631927  summer
                     2041-06-01   298.005951   298.940857   299.674408  summer
                     2071-06-01   299.187775 

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



fall
                                 tg_mean_p10  tg_mean_p50  tg_mean_p90 season
lat       lon        time                                                    
66.623306 -89.045212 1981-09-01   262.421539   262.786377   263.295746   fall
                     2011-09-01   263.786835   264.912811   265.624512   fall
                     2041-09-01   265.792114   267.586853   269.512817   fall
                     2071-09-01   268.093353   270.263397   273.660858   fall
          -88.961884 1981-09-01   262.533844   262.898529   263.405365   fall
...                                      ...          ...          ...    ...
40.041039 -74.129143 2071-09-01   290.489410   291.886261   292.117767   fall
          -74.045807 1981-09-01   286.452576   286.639282   286.785828   fall
                     2011-09-01   287.203979   288.179260   288.400360   fall
                     2041-09-01   288.621002   289.746277   290.111542   fall
                     2071-09-01   290.571228   291.952362  

In [181]:
if (var == "tg_mean" or var== "tn_mean" or var== "tx_mean"):
    dfbig[var+"_p10"] = dfbig[var+"_p10"] -273.15
    dfbig[var+"_p50"] = dfbig[var+"_p50"] -273.15
    dfbig[var+"_p90"] = dfbig[var+"_p90"] -273.15
if (var == "solidprcptot"):
    dfbig[var+"_p10"] = dfbig[var+"_p10"] /10
    dfbig[var+"_p50"] = dfbig[var+"_p50"] /10
    dfbig[var+"_p90"] = dfbig[var+"_p90"] /10
dfbig

,lat,lon,time,tg_mean_p10,tg_mean_p50,tg_mean_p90,season,scenario
0,66.623306,-89.045212,1981,-30.90,-30.26,-29.93,winter,45
1,66.623306,-89.045212,2011,-29.41,-28.30,-27.61,winter,45
2,66.623306,-89.045212,2041,-28.02,-25.82,-24.78,winter,45
3,66.623306,-89.045212,2071,-27.18,-25.61,-22.94,winter,45
4,66.623306,-88.961884,1981,-30.82,-30.17,-29.85,winter,45
...,...,...,...,...,...,...,...,...
3048235,40.041039,-74.129143,2071,15.35,16.97,17.32,annual,85
3048236,40.041039,-74.045807,1981,11.94,12.16,12.23,annual,85
3048237,40.041039,-74.045807,2011,12.96,13.35,13.68,annual,85
3048238,40.041039,-74.045807,2041,14.06,15.07,15.20,annual,85


In [190]:
for region, (name, short) in short_dict.items():
    dfregion = pd.read_feather("/home/mlopez/EXEC/Grids-polygons-regions/"+region+"lat_lon.feather")
    df_per_region = pd.merge(dfregion, dfbig.reset_index(), on=["lat","lon"])
    df_subregion_means = df_per_region.groupby(["time", name, "season","scenario"]).mean()[[var+"_p10",var+"_p50", var+"_p90"]]
    df_subregion_means[var+"_p10"] = round(df_subregion_means[var+"_p10"],2)
    df_subregion_means[var+"_p50"] = round(df_subregion_means[var+"_p50"],2)
    df_subregion_means[var+"_p90"] = round(df_subregion_means[var+"_p90"],2)
    df_subregion_means[var+"_p1090"] = df_subregion_means[var+"_p10"].map(str)+" - "+df_subregion_means[var+"_p90"].map(str)
    df_subregion_means_perc = pd.wide_to_long(df_subregion_means.reset_index(), var+"_p", i=("time","season","scenario",name), j="percentility")
    df_subregion_means_perc = df_subregion_means_perc.rename(columns={var+"_p":var })
    for (sr,df) in df_subregion_means_perc.reset_index().groupby(name):
        df = df.copy()
        df["sp"] = df.season + df.percentility.map(str)
        df["time_scenario"] = df.time.map(str) + df.scenario.map(str)    
        dftable = pd.pivot(df,columns="time_scenario", index="sp", values=var)
        print ("9/11 Reindexing table")
        dftable2 = dftable.reindex(index=["annual50", "annual1090","winter50","winter1090","spring50","spring1090",
                           "summer50","summer1090","fall50","fall1090"])
        dftable2.index=["Annuel", "Percentiles (10 - 90)","Hiver","Percentiles (10 - 90)","Printemps","Percentiles (10 - 90)",
                           "Ete","Percentiles (10 - 90)","Automne","Percentiles (10 - 90)"]
        print("10/11 Getting mean for historic data")
        dftable2["1981-2010"] = ((strings_to_nan(dftable2.iloc[:,0]) + strings_to_nan(dftable2.iloc[:,1])) / 2).fillna("")
        dftable3 = dftable2.drop(columns=["198145","198185",'201145', '201185'])
        dftable4 = dftable3[["1981-2010", "204145","204185", "207145", "207185"]]
        dftable5 = dftable4.rename(columns={'204145': 'Emissions moderees 2050',
                                            '204185': 'Emissions Elevees 2050',
                                            '207145': 'Emissions moderees 2080',
                                            '207185': 'Emissions Elevees 2080', })
        print("11/11 ... CREATING CSV!!")
        dftable5.to_csv(replace_all(sr+"_"+var+"_table.csv", d))

9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATIN

9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATIN

9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATIN

In [189]:
dftable5

time_scenario,1981-2010,Emissions moderees 2050,Emissions Elevees 2050,Emissions moderees 2080,Emissions Elevees 2080
Annuel,2.11,4.58,5.31,5.15,7.96
Percentiles (10 - 90),,3.52 - 5.32,4.64 - 6.12,3.87 - 6.19,6.03 - 9.15
Hiver,-13.83,-11.09,-9.77,-10.29,-6.86
Percentiles (10 - 90),,-11.95 - -9.74,-10.81 - -8.46,-11.33 - -8.72,-7.99 - -4.95
Printemps,1.475,3.32,3.97,4.2,5.86
Percentiles (10 - 90),,2.31 - 4.43,3.61 - 5.04,3.13 - 5.33,5.42 - 7.72
Ete,16.275,18.44,19.27,19.23,22.01
Percentiles (10 - 90),,17.83 - 19.27,18.67 - 20.64,18.06 - 19.95,20.33 - 23.64
Automne,4.27,6.49,7.16,7.14,9.9
Percentiles (10 - 90),,5.63 - 7.55,6.31 - 8.57,5.56 - 8.08,7.74 - 11.16


In [139]:
df_subregion_means_perc

tg_mean
time       season scenario NM_REG_FOR               percentility                                        
1981-01-01 annual 45       ABITIBI-TEMISCAMINGUE    10                                            275.11
                                                    50                                           275.293
                                                    90                                            275.35
                                                    1090            275.109760759346 - 275.3502572607324
                  85       ABITIBI-TEMISCAMINGUE    10                                           275.128
...                                                                                                  ...
2071-12-01 winter 45       SAGUENAY -LAC-SAINT-JEAN 1090           259.0896841805326 - 261.7599387780082
                  85       SAGUENAY -LAC-SAINT-JEAN 10                                           262.431
                                                    50                                           264.109
                                                    90                                           264.633
                                                    1090          262.4313321317319 - 264.63251014655833

[1920 rows x 1 columns]

In [57]:
for region, (name, short) in short_dict.items():
    print("------------------------------------------------------------------")
    print ("1/11... Cutting by region")
    print(region, name)
    dfregion = pd.read_feather("/home/mlopez/EXEC/Grids-polygons-regions/"+region+"lat_lon.feather")
    print ("2/11... Merging region with data")
    dfdatareg45 = pd.merge(dfregion, dfperT45.reset_index(), on=["lat","lon"])
    dfdatareg85 = pd.merge(dfregion, dfperT85.reset_index(), on=["lat","lon"])
    dfdatareg45s = pd.merge(dfregion, dfseason45.reset_index(), on=["lat","lon"])
    dfdatareg85s = pd.merge(dfregion, dfseason85.reset_index(), on=["lat","lon"])
    print ("3/11... Get mean by year for every sub-region")
    dfmean45UA = dfdatareg45.groupby(["time", name, "season"]).mean()
    dfmean45UA2 = dfmean45UA.reset_index()
    dfmean85UA = dfdatareg85.groupby(["time", name, "season"]).mean()
    dfmean85UA2 = dfmean85UA.reset_index()
    dfmean45UAs = dfdatareg45s.groupby(["time", name, "season"]).mean()
    dfmean45UA2s = dfmean45UAs.reset_index()
    dfmean85UAs = dfdatareg85s.groupby(["time", name, "season"]).mean()
    dfmean85UA2s = dfmean85UAs.reset_index()
    print ("4/11... Round RCP45")
    if (var == "tg_mean" or var== "tn_mean" or var== "tx_mean"):
        dfmean45UA2[var+"_p10"] = dfmean45UA2[var+"_p10"]- 273.15
        dfmean45UA2[var+"_p50"] = dfmean45UA2[var+"_p50"]- 273.15
        dfmean45UA2[var+"_p90"] = dfmean45UA2[var+"_p90"]- 273.15
        dfmean45UA2s[var+"_p10"] = dfmean45UA2s[var+"_p10"]- 273.15
        dfmean45UA2s[var+"_p50"] = dfmean45UA2s[var+"_p50"]- 273.15
        dfmean45UA2s[var+"_p90"] = dfmean45UA2s[var+"_p90"]- 273.15
    if (var == "solidprcptot"):
        dfmean45UA2[var+"_p10"] = dfmean45UA2[var+"_p10"]/10
        dfmean45UA2[var+"_p50"] = dfmean45UA2[var+"_p50"]/10
        dfmean45UA2[var+"_p90"] = dfmean45UA2[var+"_p90"]/10
        dfmean45UA2s[var+"_p10"] = dfmean45UA2s[var+"_p10"]/10
        dfmean45UA2s[var+"_p50"] = dfmean45UA2s[var+"_p50"]/10
        dfmean45UA2s[var+"_p90"] = dfmean45UA2s[var+"_p90"]/10
    dfmean45UA2[var+"_p10"] = round(dfmean45UA2[var+"_p10"], 2)
    dfmean45UA2[var+"_p50"] = round(dfmean45UA2[var+"_p50"], 2)
    dfmean45UA2[var+"_p90"] = round(dfmean45UA2[var+"_p90"], 2)
    dfmean45UA2s[var+"_p10"] = round(dfmean45UA2s[var+"_p10"], 2)
    dfmean45UA2s[var+"_p50"] = round(dfmean45UA2s[var+"_p50"], 2)
    dfmean45UA2s[var+"_p90"] = round(dfmean45UA2s[var+"_p90"], 2)
    print ("5/11... Round RCP85")
    if (var == "tg_mean" or var== "tn_mean" or var== "tx_mean"):
        print (var)
        dfmean85UA2[var+"_p10"] = dfmean85UA2[var+"_p10"]- 273.15
        dfmean85UA2[var+"_p50"] = dfmean85UA2[var+"_p50"]- 273.15
        dfmean85UA2[var+"_p90"] = dfmean85UA2[var+"_p90"]- 273.15
        print (dfmean85UA2[var+"_p10"][1])
        dfmean85UA2s[var+"_p10"] = dfmean85UA2s[var+"_p10"]- 273.15
        dfmean85UA2s[var+"_p50"] = dfmean85UA2s[var+"_p50"]- 273.15
        dfmean85UA2s[var+"_p90"] = dfmean85UA2s[var+"_p90"]- 273.15
    if (var == "solidprcptot"):
        dfmean85UA2[var+"_p10"] = dfmean85UA2[var+"_p10"]/10
        dfmean85UA2[var+"_p50"] = dfmean85UA2[var+"_p50"]/10
        dfmean85UA2[var+"_p90"] = dfmean85UA2[var+"_p90"]/10
        dfmean85UA2s[var+"_p10"] = dfmean85UA2s[var+"_p10"]/10
        dfmean85UA2s[var+"_p50"] = dfmean85UA2s[var+"_p50"]/10
        dfmean85UA2s[var+"_p90"] = dfmean85UA2s[var+"_p90"]/10
    dfmean85UA2[var+"_p10"] = round(dfmean85UA2[var+"_p10"], 2)
    dfmean85UA2[var+"_p50"] = round(dfmean85UA2[var+"_p50"], 2)
    dfmean85UA2[var+"_p90"] = round(dfmean85UA2[var+"_p90"], 2)
    dfmean85UA2s[var+"_p10"] = round(dfmean85UA2s[var+"_p10"], 2)
    dfmean85UA2s[var+"_p50"] = round(dfmean85UA2s[var+"_p50"], 2)
    dfmean85UA2s[var+"_p90"] = round(dfmean85UA2s[var+"_p90"], 2)
    print("6/11 ... Merging columns 10 and 90")
    dfmean45UA2[var+"_p10_p90"] = dfmean45UA2[var+"_p10"].map(str)+" - "+dfmean45UA2[var+"_p90"].map(str)
    dfreg45 = dfmean45UA2[["time", name, var+"_p50", var+"_p10_p90", "season"]]
    dfmean85UA2[var+"_p10_p90"] = dfmean85UA2[var+"_p10"].map(str)+" - "+dfmean85UA2[var+"_p90"].map(str)
    dfreg85 = dfmean85UA2[["time", name, var+"_p50", var+"_p10_p90", "season"]]
    dfreg2 = pd.merge(dfreg45, dfreg85, on=["time",name, "season"], suffixes=('_45', '_85') )
    dfmean45UA2s[var+"_p10_p90"] = dfmean45UA2s[var+"_p10"].map(str)+" - "+dfmean45UA2s[var+"_p90"].map(str)
    dfreg45s =  dfmean45UA2s[["time", name, var+"_p50", var+"_p10_p90", "season"]]
    dfmean85UA2s[var+"_p10_p90"] = dfmean85UA2s[var+"_p10"].map(str)+" - "+dfmean85UA2s[var+"_p90"].map(str)
    dfreg85s =  dfmean85UA2s[["time", name, var+"_p50", var+"_p10_p90", "season"]]
    dfreg2s = pd.merge(dfreg45s, dfreg85s, on=["time",name, "season"], suffixes=('_45', '_85') )
    print("7/11 ... Merging Annual and Seasonal DFs")
    dfall = pd.concat((dfreg2, dfreg2s))
    print("8/11 ... Pivoting and ordering Table")
    dfreg3 = pd.wide_to_long(dfall,(var+"_p50_",var+"_p10_p90_"),i=["time",name,"season"],j="scenario")
    dfreg3b = dfreg3.copy()
    dfreg3b.columns = [var+"50", var+"1090"]
    dfreg4 = pd.wide_to_long(dfreg3b.reset_index(),var, i=("time",name,"season","scenario"), j="percentility").reset_index()
    for (sr,df) in dfreg4.reset_index().groupby(name):
        print (sr)
        df = df.copy()
        df["sp"] = df.season + df.percentility.map(str)
        df["time_scenario"] = df.time.map(str) + df.scenario.map(str)    
        df_table = pd.pivot(df,columns="time_scenario", index="sp", values=var)
        print ("9/11 Reindexing table")
        dftable2 = df_table.reindex(index=["annual50", "annual1090","winter50","winter1090","spring50","spring1090",
                           "summer50","summer1090","fall50","fall1090"])
        dftable2.index=["Annuel", "Percentiles (10 - 90)","Hiver","Percentiles (10 - 90)","Printemps","Percentiles (10 - 90)",
                           "Ete","Percentiles (10 - 90)","Automne","Percentiles (10 - 90)"]
        print("10/11 Getting mean for historic data")
        dftable2["1981-2010"] = ((strings_to_nan(dftable2.iloc[:,0]) + strings_to_nan(dftable2.iloc[:,1])) / 2).fillna("")
        dftable3 = dftable2.drop(columns=['1981-01-01 00:00:0045', '1981-01-01 00:00:0085', "2011-01-01 00:00:0045", "2011-01-01 00:00:0085" ])
        dftable4 = dftable3[["1981-2010", "2041-01-01 00:00:0045", "2041-01-01 00:00:0085", "2071-01-01 00:00:0045",
                             "2071-01-01 00:00:0085"]]
        dftable5 = dftable4.rename(columns={'2041-01-01 00:00:0045': 'Emissions moderees 2050',
                                            '2041-01-01 00:00:0085': 'Emissions Elevees 2050',
                                            '2071-01-01 00:00:0045': 'Emissions moderees 2080',
                                            '2071-01-01 00:00:0085': 'Emissions Elevees 2080', })
        print("11/11 ... CREATING CSV!!")
        dftable5.to_csv(replace_all(sr+"_"+var+"_table.csv", d))

------------------------------------------------------------------
1/11... Cutting by region
DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR
2/11... Merging region with data
3/11... Get mean by year for every sub-region
4/11... Round RCP45
5/11... Round RCP85
tg_mean
2.7827922083597514
6/11 ... Merging columns 10 and 90
7/11 ... Merging Annual and Seasonal DFs
8/11 ... Pivoting and ordering Table
ABITIBI-TEMISCAMINGUE
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
BAS-SAINT-LAURENT
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
COTE-NORD
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
GASPESIE-ILES-DE-LA-MADELEINE
9/11 Reindexing table
10/11 Getting me

9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
Toundra arctique herbacée
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
Toundra forestière
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
Érablière à bouleau jaune
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
Érablière à caryer cordiforme
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
Érablière à tilleul
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
------------------------------------------------------------------
1/11... Cutting by region
REG_ECO NOM
2/11... Merging region with data
3/11... Get mean by year for every sub-region
4/11... Round RCP45
5/11... Round RCP85
tg_mean
1.0865839723680892
6/11 ... Merging columns 10 and 90
7/11 ... Merging Annual and Seasonal DFs
8/11 ... Pivoting and ordering Table
Collines ce

3/11... Get mean by year for every sub-region
4/11... Round RCP45
5/11... Round RCP85
tg_mean
-0.594462776745388
6/11 ... Merging columns 10 and 90
7/11 ... Merging Annual and Seasonal DFs
8/11 ... Pivoting and ordering Table
Centre du Québec
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
Nord-est
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
Nord-ouest
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
Secteur métropolitain et sud
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
Sud-Ouest
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
Sud-est
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
------------------------------------------------------------------
1/11... Cutting by region
SREG_ECO NOM
2/11... Merging region with data
3/11... Get mean by year for every sub-region
4

Plaine du lac Opémisca
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
Plaine du lac Saint-Jean et du Saguenay
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
Plaines et coteaux du lac Simard
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
Îles-de-la-Madeleine
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
------------------------------------------------------------------
1/11... Cutting by region
territoire_guide TER_GUIDE
2/11... Merging region with data
3/11... Get mean by year for every sub-region
4/11... Round RCP45
5/11... Round RCP85
tg_mean
4.731968180338583
6/11 ... Merging columns 10 and 90
7/11 ... Merging Annual and Seasonal DFs
8/11 ... Pivoting and ordering Table
1a
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
2a
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING C

In [58]:
df

,index,time,TER_GUIDE,season,scenario,percentility,tg_mean,sp,time_scenario
96,96,1981-01-01,6opqr,annual,45,50,-1.96,annual50,1981-01-01 00:00:0045
97,97,1981-01-01,6opqr,annual,45,1090,-2.08 - -1.84,annual1090,1981-01-01 00:00:0045
98,98,1981-01-01,6opqr,annual,85,50,-1.99,annual50,1981-01-01 00:00:0085
99,99,1981-01-01,6opqr,annual,85,1090,-2.08 - -1.79,annual1090,1981-01-01 00:00:0085
196,196,2011-01-01,6opqr,annual,45,50,-0.9,annual50,2011-01-01 00:00:0045
...,...,...,...,...,...,...,...,...,...
1995,1995,2071-01-01,6opqr,summer,85,1090,1.69 - 5.2,summer1090,2071-01-01 00:00:0085
1996,1996,2071-01-01,6opqr,winter,45,50,1.04,winter50,2071-01-01 00:00:0045
1997,1997,2071-01-01,6opqr,winter,45,1090,-0.43 - 2.22,winter1090,2071-01-01 00:00:0045
1998,1998,2071-01-01,6opqr,winter,85,50,3.95,winter50,2071-01-01 00:00:0085


In [62]:
pd.set_option("display.max_rows", 80)

In [63]:
df

,index,time,TER_GUIDE,season,scenario,percentility,tg_mean,sp,time_scenario
96,96,1981-01-01,6opqr,annual,45,50,-1.96,annual50,1981-01-01 00:00:0045
97,97,1981-01-01,6opqr,annual,45,1090,-2.08 - -1.84,annual1090,1981-01-01 00:00:0045
98,98,1981-01-01,6opqr,annual,85,50,-1.99,annual50,1981-01-01 00:00:0085
99,99,1981-01-01,6opqr,annual,85,1090,-2.08 - -1.79,annual1090,1981-01-01 00:00:0085
196,196,2011-01-01,6opqr,annual,45,50,-0.9,annual50,2011-01-01 00:00:0045
197,197,2011-01-01,6opqr,annual,45,1090,-1.52 - -0.4,annual1090,2011-01-01 00:00:0045
198,198,2011-01-01,6opqr,annual,85,50,-0.69,annual50,2011-01-01 00:00:0085
199,199,2011-01-01,6opqr,annual,85,1090,-1.31 - 0.03,annual1090,2011-01-01 00:00:0085
296,296,2041-01-01,6opqr,annual,45,50,0.34,annual50,2041-01-01 00:00:0045
297,297,2041-01-01,6opqr,annual,45,1090,-0.51 - 1.46,annual1090,2041-01-01 00:00:0045


In [60]:
pd.pivot(df,columns="time_scenario", index="sp", values=var)

time_scenario,1981-01-01 00:00:0045,1981-01-01 00:00:0085,2011-01-01 00:00:0045,2011-01-01 00:00:0085,2041-01-01 00:00:0045,2041-01-01 00:00:0085,2071-01-01 00:00:0045,2071-01-01 00:00:0085
sp,,,,,,,,
annual1090,-2.08 - -1.84,-2.08 - -1.79,-1.52 - -0.4,-1.31 - 0.03,-0.51 - 1.46,0.33 - 2.23,-0.43 - 2.22,1.69 - 5.2
annual50,-1.96,-1.99,-0.9,-0.69,0.34,1.16,1.04,3.95
fall1090,-2.08 - -1.84,-2.08 - -1.79,-1.52 - -0.4,-1.31 - 0.03,-0.51 - 1.46,0.33 - 2.23,-0.43 - 2.22,1.69 - 5.2
fall50,-1.96,-1.99,-0.9,-0.69,0.34,1.16,1.04,3.95
spring1090,-2.08 - -1.84,-2.08 - -1.79,-1.52 - -0.4,-1.31 - 0.03,-0.51 - 1.46,0.33 - 2.23,-0.43 - 2.22,1.69 - 5.2
spring50,-1.96,-1.99,-0.9,-0.69,0.34,1.16,1.04,3.95
summer1090,-2.08 - -1.84,-2.08 - -1.79,-1.52 - -0.4,-1.31 - 0.03,-0.51 - 1.46,0.33 - 2.23,-0.43 - 2.22,1.69 - 5.2
summer50,-1.96,-1.99,-0.9,-0.69,0.34,1.16,1.04,3.95
winter1090,-2.08 - -1.84,-2.08 - -1.79,-1.52 - -0.4,-1.31 - 0.03,-0.51 - 1.46,0.33 - 2.23,-0.43 - 2.22,1.69 - 5.2


In [54]:
df_table

Saison,1981-01-01 00:00:0045,1981-01-01 00:00:0085,2011-01-01 00:00:0045,2011-01-01 00:00:0085,2041-01-01 00:00:0045,2041-01-01 00:00:0085,2071-01-01 00:00:0045,2071-01-01 00:00:0085
sp,,,,,,,,
annual1090,-2.08 - -1.84,-2.08 - -1.79,-1.52 - -0.4,-1.31 - 0.03,-0.51 - 1.46,0.33 - 2.23,-0.43 - 2.22,1.69 - 5.2
annual50,-1.96,-1.99,-0.9,-0.69,0.34,1.16,1.04,3.95
fall1090,-2.08 - -1.84,-2.08 - -1.79,-1.52 - -0.4,-1.31 - 0.03,-0.51 - 1.46,0.33 - 2.23,-0.43 - 2.22,1.69 - 5.2
fall50,-1.96,-1.99,-0.9,-0.69,0.34,1.16,1.04,3.95
spring1090,-2.08 - -1.84,-2.08 - -1.79,-1.52 - -0.4,-1.31 - 0.03,-0.51 - 1.46,0.33 - 2.23,-0.43 - 2.22,1.69 - 5.2
spring50,-1.96,-1.99,-0.9,-0.69,0.34,1.16,1.04,3.95
summer1090,-2.08 - -1.84,-2.08 - -1.79,-1.52 - -0.4,-1.31 - 0.03,-0.51 - 1.46,0.33 - 2.23,-0.43 - 2.22,1.69 - 5.2
summer50,-1.96,-1.99,-0.9,-0.69,0.34,1.16,1.04,3.95
winter1090,-2.08 - -1.84,-2.08 - -1.79,-1.52 - -0.4,-1.31 - 0.03,-0.51 - 1.46,0.33 - 2.23,-0.43 - 2.22,1.69 - 5.2


In [47]:
df

,index,time,TER_GUIDE,season,scenario,percentility,tg_mean,sp,Saison
96,96,1981-01-01,6opqr,annual,45,50,-1.96,annual50,1981-01-01 00:00:0045
97,97,1981-01-01,6opqr,annual,45,1090,-2.08 - -1.84,annual1090,1981-01-01 00:00:0045
98,98,1981-01-01,6opqr,annual,85,50,-1.99,annual50,1981-01-01 00:00:0085
99,99,1981-01-01,6opqr,annual,85,1090,-2.08 - -1.79,annual1090,1981-01-01 00:00:0085
196,196,2011-01-01,6opqr,annual,45,50,-0.9,annual50,2011-01-01 00:00:0045
...,...,...,...,...,...,...,...,...,...
1995,1995,2071-01-01,6opqr,summer,85,1090,1.69 - 5.2,summer1090,2071-01-01 00:00:0085
1996,1996,2071-01-01,6opqr,winter,45,50,1.04,winter50,2071-01-01 00:00:0045
1997,1997,2071-01-01,6opqr,winter,45,1090,-0.43 - 2.22,winter1090,2071-01-01 00:00:0045
1998,1998,2071-01-01,6opqr,winter,85,50,3.95,winter50,2071-01-01 00:00:0085
